In [1]:
cd ..

/Users/muhammadfarooq/Codebase/wood-log-detection/Pith2Bark


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Damage Dataset

In [3]:
def read_damage_min_max(damage_fname):
    df_damage = pd.read_csv(damage_fname)
    
    columns = ['min', 'max', 'mean', 'orig', 'diff_min_max', 'diff_orig_mean', 'damage_mean']
    df_damage_temp = df_damage.drop(columns=columns)
    df_damage_temp.rename(columns={'damage_min_max': 'Damage'}, inplace=True)

    df_damage_temp.reset_index(drop=True, inplace=True)

    return df_damage_temp

In [4]:
def read_damage_mean(damage_fname):
    df_damage = pd.read_csv(damage_fname)
    
    columns = ['min', 'max', 'mean', 'orig', 'diff_min_max', 'diff_orig_mean', 'damage_min_max']
    df_damage_temp = df_damage.drop(columns=columns)
    df_damage_temp.rename(columns={'damage_mean': 'Damage'}, inplace=True)

    df_damage_temp.reset_index(drop=True, inplace=True)

    return df_damage_temp

In [5]:
#damage
damage_url = "damage_dataset/damage.csv"

df_damage_min_max = read_damage_min_max(damage_url)

df_damage_mean = read_damage_mean(damage_url)

In [6]:
#STOP

In [7]:
df = df_damage_min_max

In [8]:
# set up the file paths for your image data and labels
img_size = (128, 128)

data_dir = 'damage_dataset/images'

# convert the DataFrame to a dictionary with specific column names as key and value
label_dict = {k: v for k, v in zip(df['image'], df['Damage'])}

# create empty lists to store the image data and labels
images = []
labels = []

# loop through each image file in the directory
for filename in os.listdir(data_dir):
    # load the image file and convert it to a NumPy array
    image = load_img(os.path.join(data_dir, filename), target_size=img_size)
    image = img_to_array(image)
    # preprocess the image by scaling the pixel values to be between 0 and 1
    image = image.astype('float32') / 255.0
    # add the preprocessed image and its label to the lists
    images.append(image)
    labels.append(label_dict[filename])

# convert the lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# print the shape of the data to verify that it's been loaded correctly
print('Image data shape:', images.shape) #(148, 64, 64, 3)
print('Label data shape:', labels.shape) #(148,)

Image data shape: (148, 128, 128, 3)
Label data shape: (148,)


In [9]:
#STOP

# Config

In [10]:
training_percent = 0.9

epochs, batch_size = 50, 32
kf = KFold(n_splits=3, shuffle=True, random_state=42)

early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True,
    )

METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [11]:
def heat_map_normalized(y_true, y_pred):
    
    class_names = ['Not Damage', 'Damaged']
    
    skplt.metrics.plot_confusion_matrix(y_true, y_pred,
                                        figsize=(4,3),
                                        normalize=True)
    
    # Customize axis tick labels
    plt.xticks([0,1], class_names)
    plt.yticks([0,1], class_names)
    
    plt.yticks(rotation=90)
    
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [12]:
def heat_map(_cm):
    # plot confusion matrix as heatmap
    labels = ['Not Damage', 'Damaged']
    
    # Set up the matplotlib figure
    plt.figure(figsize=(4, 3))
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    
    sns.heatmap(_cm, annot=True, cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [13]:
units = [32]
activations = ['relu', 'tanh', 'sigmoid']
kernel_size = [3, 5]
learning_rate = [0.001, 0.0001, 0.00001]
optimizer = ['Adam', 'RMSprop', 'Nadam']
dropout = [0.1, 0.2, 0.3]

In [14]:
#STOP

# Model

In [15]:
class Baseline():
    def __init__(self, _input_dim, units, activation, kernel_size, learning_rate, optimizer, dropout, metrics=METRICS):
        # Define the model architecture
        model = Sequential()
        
        # convolutional layer
        model.add(Conv2D(units, kernel_size=kernel_size, activation=activation, input_shape=_input_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))
                  
        # flatten output of conv
        model.add(Flatten())
                  
        # hidden layer
        model.add(Dense(units, activation=activation))
        model.add(Dropout(dropout))
                  
        # output layer
        model.add(Dense(1, activation='sigmoid'))

        if optimizer == 'Adam':
            ops = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'RMSprop':
            ops = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        elif optimizer == 'Nadam':
            ops = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
            
        # Compile the model
        model.compile(loss='binary_crossentropy', optimizer=ops, metrics=['accuracy'])

        self.model = model

    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size):
        history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        return history
    
    def evaluate(self, X_test, y_test):
        results = self.model.evaluate(X_test, y_test, verbose=0)
        return results

    def predict(self, X_test):
        predictions = self.model.predict(X_test)
        return predictions

    def summary(self):
        self.model.summary()
        
    def metrics_names(self):
        return self.model.metrics_names

# Prepare dataset

In [16]:
X = images
y = labels

# split the data into a holdout set and the rest for K-Fold cross-validation
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size=training_percent, random_state=42)

In [17]:
best_acc = 0.0
baseline_values = { 'units': 0, 'activation': '', 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [18]:
input_dim = X_train.shape[1:]

In [19]:
for u in units:
    for a in activations:
        for k in kernel_size:
            for l in learning_rate:
                for o in optimizer:
                    for d in dropout:
                        print('\nUnits:', u, 'Activation:', a, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                        for train_index, val_index in kf.split(X_train):
                            # split the dataset into training and validation sets for this fold
                            X_train_kf, X_val_kf = X_train[train_index], X_train[val_index]
                            y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
                            
                            model = Baseline(input_dim, u, a, k, l, o, d)

                            history = model.train(X_train_kf, y_train_kf, X_val_kf, y_val_kf, epochs, batch_size)
                            if (history.history['val_accuracy'][-1] > best_acc):
                                best_acc = history.history['val_accuracy'][-1]
                                baseline_values = { 'units': u, 'activation': a, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }


Units: 32 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: Adam Dropout: 0.1
Epoch 1/50
3/3 - 1s - loss: 1.4404 - accuracy: 0.4545 - val_loss: 1.9397 - val_accuracy: 0.5556 - 1s/epoch - 450ms/step
Epoch 2/50
3/3 - 0s - loss: 1.3740 - accuracy: 0.7727 - val_loss: 1.2649 - val_accuracy: 0.8444 - 415ms/epoch - 138ms/step
Epoch 3/50
3/3 - 0s - loss: 1.3102 - accuracy: 0.8636 - val_loss: 1.1980 - val_accuracy: 0.7778 - 405ms/epoch - 135ms/step
Epoch 4/50
3/3 - 0s - loss: 0.9836 - accuracy: 0.8523 - val_loss: 0.9701 - val_accuracy: 0.7556 - 411ms/epoch - 137ms/step
Epoch 5/50
3/3 - 0s - loss: 0.6743 - accuracy: 0.8068 - val_loss: 0.8320 - val_accuracy: 0.7556 - 413ms/epoch - 138ms/step
Epoch 6/50
3/3 - 0s - loss: 0.4283 - accuracy: 0.7955 - val_loss: 0.4294 - val_accuracy: 0.7556 - 414ms/epoch - 138ms/step
Epoch 7/50
3/3 - 0s - loss: 0.5121 - accuracy: 0.8182 - val_loss: 0.3891 - val_accuracy: 0.8667 - 409ms/epoch - 136ms/step
Epoch 8/50
3/3 - 0s - loss: 0.4070 - accuracy: 0.

Epoch 9/50
3/3 - 0s - loss: 0.3618 - accuracy: 0.8864 - val_loss: 0.3957 - val_accuracy: 0.8222 - 402ms/epoch - 134ms/step
Epoch 10/50
3/3 - 0s - loss: 0.3350 - accuracy: 0.8636 - val_loss: 0.3901 - val_accuracy: 0.8667 - 397ms/epoch - 132ms/step
Epoch 11/50
3/3 - 0s - loss: 0.3371 - accuracy: 0.8750 - val_loss: 0.3946 - val_accuracy: 0.8667 - 401ms/epoch - 134ms/step
Epoch 12/50
3/3 - 0s - loss: 0.3425 - accuracy: 0.8864 - val_loss: 0.3912 - val_accuracy: 0.8667 - 396ms/epoch - 132ms/step
Epoch 13/50
3/3 - 0s - loss: 0.3047 - accuracy: 0.8636 - val_loss: 0.3971 - val_accuracy: 0.8000 - 409ms/epoch - 136ms/step
Epoch 14/50
3/3 - 0s - loss: 0.2676 - accuracy: 0.9091 - val_loss: 0.3757 - val_accuracy: 0.8667 - 404ms/epoch - 135ms/step
Epoch 15/50
3/3 - 0s - loss: 0.2397 - accuracy: 0.9205 - val_loss: 0.3569 - val_accuracy: 0.8667 - 409ms/epoch - 136ms/step
Epoch 16/50
3/3 - 0s - loss: 0.2412 - accuracy: 0.9091 - val_loss: 0.3684 - val_accuracy: 0.8667 - 397ms/epoch - 132ms/step
Epoch 17/

Epoch 8/50
3/3 - 0s - loss: 0.4827 - accuracy: 0.8636 - val_loss: 0.4845 - val_accuracy: 0.7556 - 397ms/epoch - 132ms/step
Epoch 9/50
3/3 - 0s - loss: 0.4286 - accuracy: 0.8636 - val_loss: 0.4714 - val_accuracy: 0.7556 - 397ms/epoch - 132ms/step
Epoch 10/50
3/3 - 0s - loss: 0.4196 - accuracy: 0.8409 - val_loss: 0.4808 - val_accuracy: 0.7778 - 408ms/epoch - 136ms/step
Epoch 11/50
3/3 - 0s - loss: 0.4360 - accuracy: 0.8409 - val_loss: 0.4773 - val_accuracy: 0.8000 - 395ms/epoch - 132ms/step
Epoch 12/50
3/3 - 0s - loss: 0.4423 - accuracy: 0.8636 - val_loss: 0.4728 - val_accuracy: 0.8000 - 396ms/epoch - 132ms/step
Epoch 13/50
3/3 - 0s - loss: 0.3830 - accuracy: 0.8636 - val_loss: 0.4690 - val_accuracy: 0.7556 - 398ms/epoch - 133ms/step
Epoch 14/50
3/3 - 0s - loss: 0.3793 - accuracy: 0.8523 - val_loss: 0.4557 - val_accuracy: 0.7556 - 406ms/epoch - 135ms/step
Epoch 15/50
3/3 - 0s - loss: 0.3364 - accuracy: 0.8636 - val_loss: 0.4333 - val_accuracy: 0.7556 - 410ms/epoch - 137ms/step
Epoch 16/5

Epoch 36/50
3/3 - 0s - loss: 0.0359 - accuracy: 0.9888 - val_loss: 0.3311 - val_accuracy: 0.9091 - 431ms/epoch - 144ms/step
Epoch 37/50
3/3 - 0s - loss: 0.0287 - accuracy: 1.0000 - val_loss: 0.3319 - val_accuracy: 0.9091 - 421ms/epoch - 140ms/step
Epoch 38/50
3/3 - 0s - loss: 0.0911 - accuracy: 0.9326 - val_loss: 0.3319 - val_accuracy: 0.9091 - 424ms/epoch - 141ms/step

Units: 32 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: RMSprop Dropout: 0.1
Epoch 1/50
3/3 - 1s - loss: 3.1768 - accuracy: 0.6136 - val_loss: 0.4100 - val_accuracy: 0.8222 - 979ms/epoch - 326ms/step
Epoch 2/50
3/3 - 0s - loss: 0.4627 - accuracy: 0.8068 - val_loss: 0.4077 - val_accuracy: 0.8444 - 443ms/epoch - 148ms/step
Epoch 3/50
3/3 - 0s - loss: 0.4726 - accuracy: 0.7841 - val_loss: 0.3692 - val_accuracy: 0.8667 - 436ms/epoch - 145ms/step
Epoch 4/50
3/3 - 0s - loss: 0.4004 - accuracy: 0.8409 - val_loss: 0.7664 - val_accuracy: 0.7556 - 446ms/epoch - 149ms/step
Epoch 5/50
3/3 - 0s - loss: 0.5768 - acc

Epoch 7/50
3/3 - 0s - loss: 0.4604 - accuracy: 0.8523 - val_loss: 0.4677 - val_accuracy: 0.7556 - 434ms/epoch - 145ms/step
Epoch 8/50
3/3 - 0s - loss: 0.4434 - accuracy: 0.8636 - val_loss: 0.4654 - val_accuracy: 0.8000 - 440ms/epoch - 147ms/step
Epoch 9/50
3/3 - 0s - loss: 0.3876 - accuracy: 0.8864 - val_loss: 0.4059 - val_accuracy: 0.8444 - 434ms/epoch - 145ms/step
Epoch 10/50
3/3 - 0s - loss: 0.4912 - accuracy: 0.8523 - val_loss: 0.6027 - val_accuracy: 0.7556 - 444ms/epoch - 148ms/step
Epoch 11/50
3/3 - 0s - loss: 0.5645 - accuracy: 0.8295 - val_loss: 0.6666 - val_accuracy: 0.6000 - 436ms/epoch - 145ms/step
Epoch 12/50
3/3 - 0s - loss: 0.5865 - accuracy: 0.7500 - val_loss: 0.4050 - val_accuracy: 0.8667 - 431ms/epoch - 144ms/step
Epoch 13/50
3/3 - 0s - loss: 0.4431 - accuracy: 0.8750 - val_loss: 0.4572 - val_accuracy: 0.7556 - 440ms/epoch - 147ms/step
Epoch 14/50
3/3 - 0s - loss: 0.4205 - accuracy: 0.8636 - val_loss: 0.9504 - val_accuracy: 0.7556 - 423ms/epoch - 141ms/step
Epoch 1/50


...

Epoch 42/50
3/3 - 1s - loss: 0.6281 - accuracy: 0.6067 - val_loss: 0.5362 - val_accuracy: 0.9091 - 534ms/epoch - 178ms/step
Epoch 43/50
3/3 - 1s - loss: 0.6315 - accuracy: 0.6292 - val_loss: 0.4907 - val_accuracy: 0.7500 - 541ms/epoch - 180ms/step
Epoch 44/50
3/3 - 1s - loss: 0.6248 - accuracy: 0.6742 - val_loss: 0.5362 - val_accuracy: 0.8636 - 535ms/epoch - 178ms/step
Epoch 45/50
3/3 - 1s - loss: 0.5826 - accuracy: 0.7528 - val_loss: 0.4881 - val_accuracy: 0.7955 - 535ms/epoch - 178ms/step
Epoch 46/50
3/3 - 1s - loss: 0.6658 - accuracy: 0.6067 - val_loss: 0.5782 - val_accuracy: 0.8636 - 522ms/epoch - 174ms/step
Epoch 47/50
3/3 - 1s - loss: 0.6090 - accuracy: 0.6517 - val_loss: 0.6270 - val_accuracy: 0.7955 - 542ms/epoch - 181ms/step
Epoch 48/50
3/3 - 1s - loss: 0.5944 - accuracy: 0.7303 - val_loss: 0.4727 - val_accuracy: 0.7500 - 536ms/epoch - 179ms/step
Epoch 49/50
3/3 - 1s - loss: 0.6163 - accuracy: 0.6292 - val_loss: 0.6229 - val_accuracy: 0.7955 - 526ms/epoch - 175ms/step
Epoch 50

Epoch 8/50
3/3 - 1s - loss: 0.6447 - accuracy: 0.6742 - val_loss: 0.6926 - val_accuracy: 0.5000 - 514ms/epoch - 171ms/step
Epoch 9/50
3/3 - 1s - loss: 0.6150 - accuracy: 0.6854 - val_loss: 0.7639 - val_accuracy: 0.5000 - 546ms/epoch - 182ms/step
Epoch 10/50
3/3 - 1s - loss: 0.6418 - accuracy: 0.6742 - val_loss: 0.7089 - val_accuracy: 0.5000 - 531ms/epoch - 177ms/step
Epoch 11/50
3/3 - 1s - loss: 0.6043 - accuracy: 0.6854 - val_loss: 0.8150 - val_accuracy: 0.5000 - 536ms/epoch - 179ms/step
Epoch 12/50
3/3 - 1s - loss: 0.6475 - accuracy: 0.6854 - val_loss: 0.8266 - val_accuracy: 0.5000 - 545ms/epoch - 182ms/step
Epoch 1/50
3/3 - 1s - loss: 0.9043 - accuracy: 0.4157 - val_loss: 0.5853 - val_accuracy: 0.7500 - 1s/epoch - 356ms/step
Epoch 2/50
3/3 - 1s - loss: 0.7527 - accuracy: 0.3820 - val_loss: 0.6168 - val_accuracy: 0.7500 - 524ms/epoch - 175ms/step
Epoch 3/50
3/3 - 1s - loss: 0.7437 - accuracy: 0.5730 - val_loss: 0.6302 - val_accuracy: 0.7500 - 541ms/epoch - 180ms/step
Epoch 4/50
3/3 -

Epoch 47/50
3/3 - 0s - loss: 0.5617 - accuracy: 0.7500 - val_loss: 0.5897 - val_accuracy: 0.7778 - 485ms/epoch - 162ms/step
Epoch 48/50
3/3 - 0s - loss: 0.5538 - accuracy: 0.7841 - val_loss: 0.5898 - val_accuracy: 0.7556 - 473ms/epoch - 158ms/step
Epoch 49/50
3/3 - 1s - loss: 0.5626 - accuracy: 0.7727 - val_loss: 0.5858 - val_accuracy: 0.7556 - 502ms/epoch - 167ms/step
Epoch 50/50
3/3 - 0s - loss: 0.5668 - accuracy: 0.7727 - val_loss: 0.5840 - val_accuracy: 0.7556 - 484ms/epoch - 161ms/step
Epoch 1/50
3/3 - 1s - loss: 0.6826 - accuracy: 0.5843 - val_loss: 0.7335 - val_accuracy: 0.5000 - 1s/epoch - 479ms/step
Epoch 2/50
3/3 - 0s - loss: 0.6349 - accuracy: 0.6854 - val_loss: 0.8451 - val_accuracy: 0.5000 - 480ms/epoch - 160ms/step
Epoch 3/50
3/3 - 0s - loss: 0.6542 - accuracy: 0.6966 - val_loss: 0.8597 - val_accuracy: 0.5000 - 485ms/epoch - 162ms/step
Epoch 4/50
3/3 - 1s - loss: 0.6298 - accuracy: 0.6966 - val_loss: 0.8005 - val_accuracy: 0.5000 - 522ms/epoch - 174ms/step
Epoch 5/50
3/3 

Epoch 31/50
3/3 - 0s - loss: 0.6390 - accuracy: 0.6591 - val_loss: 0.6188 - val_accuracy: 0.6444 - 489ms/epoch - 163ms/step
Epoch 32/50
3/3 - 1s - loss: 0.6459 - accuracy: 0.6591 - val_loss: 0.6151 - val_accuracy: 0.6444 - 504ms/epoch - 168ms/step
Epoch 33/50
3/3 - 1s - loss: 0.5913 - accuracy: 0.7045 - val_loss: 0.6125 - val_accuracy: 0.6444 - 502ms/epoch - 167ms/step
Epoch 34/50
3/3 - 0s - loss: 0.6682 - accuracy: 0.6136 - val_loss: 0.6107 - val_accuracy: 0.6444 - 494ms/epoch - 165ms/step
Epoch 35/50
3/3 - 1s - loss: 0.6130 - accuracy: 0.6705 - val_loss: 0.6091 - val_accuracy: 0.6444 - 503ms/epoch - 168ms/step
Epoch 36/50
3/3 - 0s - loss: 0.6136 - accuracy: 0.6477 - val_loss: 0.6069 - val_accuracy: 0.6444 - 495ms/epoch - 165ms/step
Epoch 37/50
3/3 - 1s - loss: 0.6175 - accuracy: 0.6705 - val_loss: 0.6057 - val_accuracy: 0.6444 - 503ms/epoch - 168ms/step
Epoch 38/50
3/3 - 0s - loss: 0.5978 - accuracy: 0.7045 - val_loss: 0.6085 - val_accuracy: 0.7556 - 497ms/epoch - 166ms/step
Epoch 39

Epoch 17/50
3/3 - 1s - loss: 0.6188 - accuracy: 0.6364 - val_loss: 0.6320 - val_accuracy: 0.6444 - 548ms/epoch - 183ms/step
Epoch 18/50
3/3 - 1s - loss: 0.6546 - accuracy: 0.6705 - val_loss: 0.6304 - val_accuracy: 0.6444 - 524ms/epoch - 175ms/step
Epoch 19/50
3/3 - 1s - loss: 0.6514 - accuracy: 0.5682 - val_loss: 0.6290 - val_accuracy: 0.6444 - 540ms/epoch - 180ms/step
Epoch 20/50
3/3 - 1s - loss: 0.6585 - accuracy: 0.6136 - val_loss: 0.6299 - val_accuracy: 0.6444 - 521ms/epoch - 174ms/step
Epoch 21/50
3/3 - 1s - loss: 0.6163 - accuracy: 0.6705 - val_loss: 0.6270 - val_accuracy: 0.6444 - 533ms/epoch - 178ms/step
Epoch 22/50
3/3 - 1s - loss: 0.6370 - accuracy: 0.6136 - val_loss: 0.6245 - val_accuracy: 0.6444 - 540ms/epoch - 180ms/step
Epoch 23/50
3/3 - 1s - loss: 0.6350 - accuracy: 0.6250 - val_loss: 0.6227 - val_accuracy: 0.6444 - 528ms/epoch - 176ms/step
Epoch 24/50
3/3 - 1s - loss: 0.6208 - accuracy: 0.6477 - val_loss: 0.6193 - val_accuracy: 0.6444 - 528ms/epoch - 176ms/step
Epoch 25

In [20]:
print(best_acc)
print(baseline_values)

0.9090909361839294
{'units': 32, 'activation': 'relu', 'kernel_size': 3, 'learning_rate': 0.001, 'optimizer': 'Adam', 'dropout': 0.1}


In [21]:
#STOP